# Use Case 6 - Teleconnection explorer

## Problem definition

As part of a project on climatic teleconnection, a student investigates how El Niño-Southern Oscillation (ENSO) relates to monsoon rainfall. A result could be a plot showing the sliding correlation between Indian Summer Monsoon Rainfall (ISMR) and SST anomalies [RD-4]. A more sophisticated version of this task would be to calculate the Multivariate ENSO Index (MEI, [RD-5], [RD-6]). Additionally, also the comparison of the ENSO index with other CCI datasets (e.g. Cloud, Fire) would be interesting.

## Required toolbox features

- Access to and ingestion of ESA CCI SST and Soil Moisture data
- Geometric adjustments
- Spatial and temporal filtering and subsetting for both data sets
- Calculation of anomalies and statistical quantities
- Visual presentation of statistical results and time series
- ENSO index calculation from SST data
- Calculation of the absolute anomaly on the log transformed soil moisture data
- Calculation of the correlation between the two time series with a lag of 30 days
- Generation of a correlation map and export of the correlation data
- Generation of a time series plot of the correlation by the selection of a location in South East Asia
- Saving of the image and the underlying data

# Ingest data, spatial and temporal subsetting, variable selection

To reduce the amount of data to download and later work with, it is beneficial to do spatial and temporal subsetting, as well as variable selection as early in the workflow pipeline as possible. Cate supports these operations at download time, or when opening a dataset.

In [1]:
from cate.core.ds import DATA_STORE_REGISTRY
import cate.ops as ops
from cate.util import ConsoleMonitor

In [2]:
monitor = ConsoleMonitor()

In [3]:
data_store = DATA_STORE_REGISTRY.get_data_store('esa_cci_odp')

In [4]:
soil_sources = data_store.query('soil')
soil_sources

[esacci.SOILMOISTURE.day.L3S.SSMV.multi-sensor.multi-platform.COMBINED.02-2.r1,
 esacci.SOILMOISTURE.day.L3S.SSMV.multi-sensor.multi-platform.PASSIVE.02-2.r1,
 esacci.SOILMOISTURE.day.L3S.SSMS.multi-sensor.multi-platform.ACTIVE.02-2.r1,
 esacci.SOILMOISTURE.day.L3S.SSMV.multi-sensor.multi-platform.COMBINED.02-1.r1,
 esacci.SOILMOISTURE.day.L3S.SSMV.multi-sensor.multi-platform.PASSIVE.02-1.r1,
 esacci.SOILMOISTURE.day.L3S.SSMS.multi-sensor.multi-platform.ACTIVE.02-1.r1]

In [5]:
soil_sources[0].meta_info

OrderedDict([('cci_project', 'SOILMOISTURE'),
             ('data_type', 'SSMV'),
             ('number_of_aggregations', 0),
             ('number_of_files', 13210),
             ('platform_id', 'multi-platform'),
             ('processing_level', 'L3S'),
             ('product_string', 'COMBINED'),
             ('product_version', '02-2'),
             ('project', 'esacci'),
             ('realization', 'r1'),
             ('sensor_id', 'multi-sensor'),
             ('size', 6648846234),
             ('time_frequency', 'day'),
             ('version', '20160909'),
             ('protocols', ['HTTPServer', 'OPENDAP']),
             ('variables',
              [{'long_name': 'Day(100) or Night(110) Flag',
                'name': 'dnflag',
                'standard_name': 'day(100)_or_night(110)_flag',
                'units': 'None'},
               {'long_name': 'Flag',
                'name': 'flag',
                'standard_name': 'flag',
                'units': 'None'},
         

In [6]:
help(soil_sources[0].make_local)

Help on method make_local in module cate.ds.esa_cci_odp:

make_local(local_name:str, local_id:str=None, time_range:Union[Tuple[str, str], Tuple[datetime.datetime, datetime.datetime], Tuple[datetime.date, datetime.date], str]=None, region:Union[shapely.geometry.polygon.Polygon, List[Tuple[float, float]], str, Tuple[float, float, float, float]]=None, var_names:Union[List[str], str]=None, monitor:cate.util.monitor.Monitor=Monitor.NONE) -> 'DataSource' method of cate.ds.esa_cci_odp.EsaCciOdpDataSource instance
    Turns this (likely remote) data source into a local data source given a name and a number of
    optional constraints.
    
    If this is a remote data source, data will be downloaded and turned into a local data source which will
    be added to the data store named "local".
    
    If this is already a local data source, a new local data source will be created by copying
    required data or data subsets.
    
    The method returns the newly create local data source.
    
  

In [8]:
s_india = '72, 8, 85, 17' # 'lon_min, lat_min, lon_max, lat_max'
soil_sources[0].make_local(local_name='soil.combined.day',
                           region=s_india,
                           time_range='2007-01-01, 2010-12-31',
                           var_names='sm, sm_uncertainty',
                           monitor=monitor)

ValueError: Local data store 'local' already contains a data source named 'local.soil.combined.day'